# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [31]:
import pandas as pd #импортируем нужные библиотеки и классы
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyClassifier

In [32]:
df = pd.read_csv('/datasets/users_behavior.csv')

Откроем и изучим данные.

In [33]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Видим, что пропусков в данных нет, типы так же соблюдены


## Разбейте данные на выборки

In [34]:
train , test = train_test_split(
df,test_size=0.2, random_state=12345)

In [35]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2571 entries, 348 to 482
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     2571 non-null   float64
 1   minutes   2571 non-null   float64
 2   messages  2571 non-null   float64
 3   mb_used   2571 non-null   float64
 4   is_ultra  2571 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 120.5 KB


In [36]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 1415 to 1196
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


Разделили выборку на тестовую и обучающую.

In [ ]:
features_test = test.drop('is_ultra',axis=1)
target_test = test['is_ultra']

Выделили целевой признак в тестовой выборке

In [ ]:
features_learn = train.drop('is_ultra',axis=1)# извлечем признаки 
target_learn = train['is_ultra']# извлечем целевой признак

Выделили целевой признак в обучающей выборке

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_learn, target_learn, test_size=0.25, random_state=12345) # отделим 25% данных для валидационной выборки

Разделили обучающую выборку на обучающую и валидационную.

## Исследуйте модели

### Decision Tree

In [ ]:
best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    if best_accuracy < accuracy:
        best_accuracy = accuracy
        best_depth = depth
print("Лучшая глубина:", best_depth, "Лучшая accuracy:", best_accuracy)

### Random Forest

In [ ]:
best_accuracy = 0
best_depth = 0
best_est = 0
best_samples = 0
for depth in range (1, 11):
    for est in range(5, 51, 5):
        for sample in range(2,5):
            model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, min_samples_leaf=sample)
            model.fit(features_train, target_train)
            accuracy = model.score(features_valid, target_valid)
            if best_accuracy < accuracy:
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_samples = sample
print("Лучшая глубина:", best_depth, "Лучшая accuracy:", best_accuracy, "Лучшее количество деревьев:", best_est, 
      "Лучшее количество узлов:", best_samples)

### Logictis Regression

In [ ]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print("Accuracy:", accuracy)

Исследовали модели применяя разные гиперпараметры. На обучающей выборке себя показл лучше всего случайный лес со следующими гиперпараметрами:
- глубина - 10
- количество деревьев - 10
- минимальное количество узлов - 3

## Проверьте модель на тестовой выборке

### Decision Tree

In [ ]:
model = DecisionTreeClassifier(random_state=12345, max_depth=7)
model.fit(features_train, target_train)
accuracy_tree = model.score(features_test, target_test)
print("Accuracy:", accuracy_tree)

### Random Forest

In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=10, max_depth=10, min_samples_leaf=3)
model.fit(features_train, target_train)
accuracy_forest = model.score(features_test, target_test)
print("Accuracy:",accuracy_forest)

### Logictis Regression

In [ ]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy_regr = model.score(features_test, target_test)
print("Accuracy:", accuracy_regr)

Смогли добиться значения `accuracy` на тестовой выборке выше 0,75. Случйный лес имеет показатель в 0,792.

## (бонус) Проверьте модели на адекватность

In [ ]:
checker = DummyClassifier(strategy='most_frequent', random_state=12345)
checker.fit(features_train,target_train)
accuracy_check = checker.score(features_test, target_test)
print('Проверочная accuracy:', accuracy_check)
print('Accuracy', accuracy_forest)

Видим, что точность модели выше проверочной точности.

## Итоги

Проверили три модели: `DecisionTreeClassifier`, `RandomForestClassifier`, `Logistic regression`
Моделью с лучшей точность `0,792` получилась `RandomForestClassifier` со следуюищими гиперпараметрами:
- глубина - 10
- количество деревьев - 10
- минимальное количество узлов - 3

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 4: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
